In [6]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import os

### Loading Land Area Data

In [122]:
areaDf = pd.read_csv("data/2019_Gaz_counties_national.txt", delimiter="\t")
areaDf = areaDf.rename(columns={"GEOID":"FIPS"})
print(areaDf.shape)
areaDf.tail()

(3220, 10)


,USPS,FIPS,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
3215,PR,72145,1804553,Vega Baja Municipio,118745077,57827590,45.848,22.327,18.455128,-66.397883
3216,PR,72147,1804554,Vieques Municipio,131541395,552192819,50.788,213.203,18.125418,-65.432474
3217,PR,72149,1804555,Villalba Municipio,92298569,3622639,35.637,1.399,18.130718,-66.472244
3218,PR,72151,1804556,Yabucoa Municipio,143005186,72592516,55.215,28.028,18.059858,-65.859871
3219,PR,72153,1804557,Yauco Municipio,175371927,1625259,67.711,0.628,18.085669,-66.857901


### Loading Population and Urban Data

In [95]:
popDf = pd.read_csv("data/PopulationEstimates.csv", encoding='utf-8')
popColsRaw = ["FIPS", "State", "Area_Name", "Rural-urban_Continuum Code_2013", "POP_ESTIMATE_2018"]
popCols = ["FIPS", "State", "County", "Urban", "Population"]
popDf = popDf[popColsRaw]
popDf.columns = popCols
popDf = popDf.dropna()
popDf["Population"] = popDf["Population"].apply(lambda x: x.replace(",", "")).astype('int64')
print(popDf.shape)
popDf.tail(5)

(3220, 5)


,FIPS,State,County,Urban,Population
3268,72145,PR,"Vega Baja Municipio, Puerto Rico",1.0,50185
3269,72147,PR,"Vieques Municipio, Puerto Rico",7.0,8364
3270,72149,PR,"Villalba Municipio, Puerto Rico",2.0,21476
3271,72151,PR,"Yabucoa Municipio, Puerto Rico",1.0,32158
3272,72153,PR,"Yauco Municipio, Puerto Rico",2.0,33860


### Loading Age group data (United States)

In [131]:
ageDf = pd.read_csv("data/cc-est2018-alldata.csv")
ageDf = ageDf[ageDf["YEAR"] == 11] # 2018 data
ageDf = ageDf[ageDf["AGEGRP"] >= 14] # 65+ age group
ageDf = ageDf.groupby(["STATE", "COUNTY", "STNAME", "CTYNAME"])["TOT_POP"].sum().reset_index()
def getFips(state, county):
    return int(str(state) + '000') + county
ageDf["FIPS"] = ageDf[["STATE", "COUNTY"]].apply(lambda x: getFips(x[0], x[1]), axis=1)
ageDf = ageDf.rename(columns={"STATE":"State_FIP"})
ageDf = ageDf.drop(["COUNTY"], axis=1)
ageDf.head()

,State_FIP,STNAME,CTYNAME,TOT_POP,FIPS
0,1,Alabama,Autauga County,8653,1001
1,1,Alabama,Baldwin County,44571,1003
2,1,Alabama,Barbour County,4832,1005
3,1,Alabama,Bibb County,3690,1007
4,1,Alabama,Blount County,10548,1009


### Loading Age group data (Puerto Rico)

In [132]:
prAgeDf = pd.read_csv("data/PEP_2018_PEPAGESEX_with_ann.csv")
prColsRaw = ["GEO.id2", "GEO.display-label", "est72018sex0_age65plus"]
prCols = ["FIPS", "CTYNAME", "TOT_POP"]
prAgeDf = prAgeDf[prColsRaw].drop(0)
prAgeDf.columns = prCols
prAgeDf["STNAME"] = "Puerto Rico"
prAgeDf["State_FIP"] = 72
prAgeDf = prAgeDf[["State_FIP", "STNAME", "CTYNAME", "TOT_POP", "FIPS"]]
prAgeDf.head()

,State_FIP,STNAME,CTYNAME,TOT_POP,FIPS
1,72,Puerto Rico,"Adjuntas Municipio, Puerto Rico",3423,72001
2,72,Puerto Rico,"Aguada Municipio, Puerto Rico",7093,72003
3,72,Puerto Rico,"Aguadilla Municipio, Puerto Rico",11070,72005
4,72,Puerto Rico,"Aguas Buenas Municipio, Puerto Rico",4974,72007
5,72,Puerto Rico,"Aibonito Municipio, Puerto Rico",4660,72009


In [133]:
allAgeDf = pd.concat([ageDf, prAgeDf])
print(allAgeDf.shape)
allAgeDf.sample(5)

(3220, 5)


,State_FIP,STNAME,CTYNAME,TOT_POP,FIPS
719,18,Indiana,Fountain County,3282,18045
1276,26,Michigan,Lenawee County,18645,26091
70,2,Alaska,Bethel Census Area,1402,2050
2775,48,Texas,Zapata County,1843,48505
1252,26,Michigan,Dickinson County,5606,26043


### Calculating Elderly Fraction

In [134]:
ageReq = ["State_FIP", "FIPS", "STNAME", "TOT_POP"]
allAgeDf["FIPS"] = allAgeDf["FIPS"].astype("int64")
combinedDf = popDf.merge(allAgeDf[ageReq], on="FIPS", how='left')
combinedDf = combinedDf.rename(columns={"STNAME":"StateName", "TOT_POP":"Frac65Plus"})
combinedDf["Frac65Plus"] = combinedDf["Frac65Plus"].astype('float64')
def calculate_elderly_frac(total_pop, elderly_pop):
    return elderly_pop * 1.0 / total_pop
combinedDf["Frac65Plus"] = combinedDf[["Population", "Frac65Plus"]].apply(lambda x: calculate_elderly_frac(x[0], x[1]), axis=1)
combinedDf.head()

,FIPS,State,County,Urban,Population,State_FIP,StateName,Frac65Plus
0,1001,AL,Autauga County,2.0,55601,1,Alabama,0.155627
1,1003,AL,Baldwin County,3.0,218022,1,Alabama,0.204433
2,1005,AL,Barbour County,6.0,24881,1,Alabama,0.194204
3,1007,AL,Bibb County,1.0,22400,1,Alabama,0.164732
4,1009,AL,Blount County,1.0,57840,1,Alabama,0.182365


### Calculating Population Density

In [135]:
demoDf = combinedDf.merge(areaDf[["FIPS", "ALAND_SQMI"]], on="FIPS", how="left")
def calculate_density(pop, area):
    return pop * 1.0 / area
demoDf["PopDensity"] = demoDf[["Population", "ALAND_SQMI"]].apply(lambda x: calculate_density(x[0], x[1]),axis=1)
demoDf = demoDf.drop("ALAND_SQMI", axis=1)
demoDf.head()

,FIPS,State,County,Urban,Population,State_FIP,StateName,Frac65Plus,PopDensity
0,1001,AL,Autauga County,2.0,55601,1,Alabama,0.155627,93.534462
1,1003,AL,Baldwin County,3.0,218022,1,Alabama,0.204433,137.136021
2,1005,AL,Barbour County,6.0,24881,1,Alabama,0.194204,28.113870
3,1007,AL,Bibb County,1.0,22400,1,Alabama,0.164732,35.986190
4,1009,AL,Blount County,1.0,57840,1,Alabama,0.182365,89.697921


In [136]:
demoDf.columns

Index(['FIPS', 'State', 'County', 'Urban', 'Population', 'State_FIP',
       'StateName', 'Frac65Plus', 'PopDensity'],
      dtype='object')

In [137]:
colsOrder = ['FIPS', 'State_FIP', 'State', 'StateName', 'County', 'Urban', 'Population', 'Frac65Plus', 'PopDensity']
demoDf = demoDf[colsOrder]
demoDf.head()

,FIPS,State_FIP,State,StateName,County,Urban,Population,Frac65Plus,PopDensity
0,1001,1,AL,Alabama,Autauga County,2.0,55601,0.155627,93.534462
1,1003,1,AL,Alabama,Baldwin County,3.0,218022,0.204433,137.136021
2,1005,1,AL,Alabama,Barbour County,6.0,24881,0.194204,28.113870
3,1007,1,AL,Alabama,Bibb County,1.0,22400,0.164732,35.986190
4,1009,1,AL,Alabama,Blount County,1.0,57840,0.182365,89.697921


In [138]:
demoDf.to_csv("data/us_demography.csv", index=False)

In [156]:
demoDf[demoDf["State"] == 'DC']

,FIPS,State_FIP,State,StateName,County,Urban,Population,Frac65Plus,PopDensity
319,11001,11,DC,District of Columbia,District of Columbia,1.0,702455,0.121436,11490.038602
